# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from null_behaviors import find_best_arc, curvature_utils, curv_of_traj_utils, opt_arc_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0402"

In [ ]:
reload(opt_arc_utils)

In [ ]:
reduce_y_var_lags = True
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = planning_and_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

In [ ]:
stop!

# CCA

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## check data

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore')
X2 = pn.y_var_reduced.drop(columns=['bin', 'segment'], errors='ignore')

X1_lags = pn.x_var_lags_reduced[[col for col in pn.x_var_lags_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore')
X2_lags = pn.y_var_lags_reduced.drop(columns=['bin', 'segment'], errors='ignore')

In [ ]:
vif_df = drop_high_vif_vars.get_vif_df(X1)
print('X1 vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X2)
print('X2 vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X1_lags)
print('X1_lags vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X2_lags)
print('X2_lags vif_df: ', vif_df.head(7))

In [ ]:
X1_lags

In [ ]:
vif_df = drop_high_vif_vars.get_vif_df(X2)
vif_df.tail(10)

## conduct cca

In [ ]:
# If error occurs, try reducing y_var again, and can also try pn.y_var_reduced.corr()[pn.y_var_reduced.corr() > 0.9]


X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var_reduced.drop(columns=['bin', 'segment'], errors='ignore').copy()
cca_no_lag = cca_class.CCAclass(X1=X1, 
                                X2=X2, 
                                lagging_included=False)
cca_no_lag.conduct_cca()

X1_lags = pn.x_var_lags_reduced[[col for col in pn.x_var_lags_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2_lags = pn.y_var_lags_reduced.drop(columns=['bin', 'segment'], errors='ignore').copy()
cca_lags = cca_class.CCAclass(X1=X1_lags, 
                              X2=X2_lags, 
                              lagging_included=True)
# for all columns that end with _0, rename them to the column name without the _0
cca_lags.X2.columns = cca_lags.X2.columns.str.replace('_0', '')
cca_lags.conduct_cca()


print(f'pn.x_var_lags.shape: {pn.x_var_lags.shape}')
print(f'pn.y_var_lags_reduced.shape: {pn.y_var_lags_reduced.shape}')

cca_inst = cca_lags